This notebook prepares Gromacs simulation files using parameters from FB Buckingham optimization, and information ported from my Amber calculations.

In [24]:
import parmed as pmd

%cd /home/winter/onsager/gromacs/
for i in range(1,21):
    parm = pmd.load_file('/home/winter/onsager/amber/Occboxtwo'+str(i)+'.prmtop',
                         '/home/winter/onsager/amber/Occboxtwo'+str(i)+'.inpcrd')
    parm.save('meta'+str(i)+'.top', format='gromacs')
    parm.save('meta'+str(i)+'.gro')

/home/winter/onsager/gromacs


Then, change the last line of `.gro` to increase box size from 4 to 4.1nm, otherwise minimization may give infinitely large gradient and very negative energy which I guess is due to Buckingham potential of two too-close solvent molecules across periodic boundary.

In [25]:
for i in range(1,21):
    !head -n -1 meta{i}.gro > temp
    !mv temp meta{i}.gro
    !echo "   4.10000   4.10000   4.10000" >> meta{i}.gro

Write the vdw parameters below to file `template.top`. All the lines except h2 line and cl line are from FB optimized `/home/winter/onsager/fb/fbO_allboxbuckB4smallerprior/input.tmp/scan_dihrot/iter_0012/fbO.itp`.
h2 line parameters are copied from hc line; cl line parameters are taken from Mayo paper table 2 without optimization.

```
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
2 1 no 0.5 0.83333333

[ atomtypes ]
; name    at.num    mass    charge ptype  a b c
ca             6  12.010000  0.00000000  A  369726  3.727249595168e+01  0.00244 ; PRM 6
os             8  16.000000  0.00000000  A  230235  39.6  0.00112
cc             6  12.010000  0.00000000  A  369726  3.788429621306e+01  0.00244 ; PRM 6
cf             6  12.010000  0.00000000  A  369726  3.766812016843e+01  0.00244 ; PRM 6
c3             6  12.010000  0.00000000  A  369726  3.803089983960e+01  0.00244 ; PRM 6
ha             1   1.008000  0.00000000  A  14258  4.112242025638e+01  0.000131 ; PRM 6
hc             1   1.008000  0.00000000  A  14258  4.064461709439e+01  0.000131 ; PRM 6
h2             1   1.008000  0.00000000  A  14258  4.064461709439e+01  0.000131
cl            17  35.450000  0.00000000  A  946820 35.09 0.00794
```

Next, assemble a .top file using 
1. Sections from the .top created above 
2. Sections from FB optimized .itp file
3. Text stored in file `templatelastlines.top`

In [26]:
for i in range(1,21):
    !cp template.top temp
    !head -n 62 meta{i}.top | tail -n 30 >> temp
    !tail -n +15 /home/winter/onsager/fb/fbO_allboxbuckB4smallerprior/input.tmp/scan_dihrot/iter_0012/fbO.itp >> temp
    !cat templatelastlines.top >> temp
    !mv temp B4smaller{i}.top

Now switch to TSCC cuz Gromacs installed there.
The job submission scripts: (the protocols are analogous to my Amber scripts of the same name)
`gromacs/min.sh` `gromacs/resheat.sh` `gromacs/reseq.sh` `gromacs/prod.sh` `gromacs/prodvulcan.sh`
Note: prod.sh doesn't use GPU for production phase, cuz Gromacs GPU requires Verlet, while Verlet doesn't support Buckingham.

A non-zero charge warning may abort minimization. In such case, manually correct the charge rounding by editing the FB optimized .itp, and go back to rerun the last cell.

In order to do restrained heating and equilibration, I created a positional restraint file `posre.itp` by `gmx genrestr -f test.gro` and following prompt. This restrain file should work for any FF parameters and any initial structures because the only information contained in it are atom numbering and restraint strength.

Successfulness of equilibration can be checked with 
`gmx energy -f B4smaller_reseq11.edr -o density.xvg` and following prompt.